# baseline 코드 이해하기

이 코드는 OpenAI의 GPT 모델을 활용하여 프롬프트 기반의 대화형 응답을 생성하는 Python 스크립트. 환경 변수와 프롬프트를 외부 파일에서 불러와서, 지정된 프롬프트에 따라 GPT-4o 모델을 호출하고 결과를 반환하는 구조.

구체적인 흐름은 다음과 같음:

- 환경 변수 로드: dotenv를 이용해 .env 파일에서 OPENAI_API_KEY 등 환경 변수를 불러오는 단계
- OpenAI 클라이언트 초기화: 불러온 API 키로 OpenAI 클라이언트 객체를 생성하는 단계
- 프롬프트 로드: load_prompts 함수에서 YAML 파일(code/prompts.yaml)로부터 system/user 프롬프트를 읽어오는 단계
- GPT 호출: call_gpt 함수에서 지정된 프롬프트 키에 따라 system/user 프롬프트를 선택하고, OpenAI의 chat.completions.create 메서드로 GPT-4o 모델을 호출하는 단계
- 예외 처리: GPT 호출 과정에서 예외가 발생할 경우 에러 메시지를 출력하고 None을 반환하는 단계
- 메인 실행: 스크립트가 직접 실행될 때 call_gpt("default")로 기본 프롬프트에 대한 응답을 받아 출력하는 단계

전체적으로, 외부 프롬프트와 환경 변수 기반으로 GPT-4o 모델을 호출하여 대화형 응답을 생성하는 자동화 흐름.


---

# LLM이란?

## 1. **LLM의 정의**

**LLM**은 **Large Language Model**의 약자로, **대규모 언어 모델**이라는 의미를 가짐.

수십억 개 이상의 파라미터(매개변수)를 가지며, **자연어 처리(NLP)** 작업을 수행할 수 있는 인공지능 모델을 지칭함.

## 2. **기본 개념**

### 2.1. 언어 모델(Language Model)이란?

- 문장 또는 단어의 **등장 확률**을 예측하는 모델
- 예: “오늘 날씨가 ___”라는 문장에서 빈칸에 들어갈 단어를 예측하는 것
- 초기에는 **통계 기반 N-gram 모델**, 이후에는 **딥러닝 기반 RNN, LSTM**, 그리고 최근에는 **Transformer 기반 모델**로 발전함

### 2.2. LLM은 무엇이 'Large'한가?

- 학습에 사용된 **데이터 규모**가 수백 GB ~ 수 PB 단위
- 모델을 구성하는 **파라미터 수**가 수십억(Billion) ~ 수천억(Trillion) 단위
- 처리 가능한 **문맥의 길이(Context Window)** 또한 수천~수만 단어에 달함

## 3. **대표적인 LLM 예시**

| 모델명 | 개발사 | 특징 |
| --- | --- | --- |
| GPT-4 | OpenAI | 대화, 글쓰기, 요약, 번역, 코딩 등 다기능 수행 가능 |
| Claude | Anthropic | 인간 중심의 안전성과 유연성을 강조 |
| Gemini (PaLM 계열) | Google | 검색 최적화와 멀티모달 연산 기능 강화 |
| LLaMA | Meta | 연구 및 오픈소스 목적의 경량화 모델 |

## 4. **LLM의 작동 원리 요약**

### 4.1. 사전학습(Pretraining)

- 대규모 텍스트 데이터를 통해 **언어 패턴과 구조를 학습**
- **자기회귀 방식(Autoregressive)** 혹은 **마스킹 방식(Masked)** 사용
- 예: “I ___ to school” → “go”라는 단어를 예측하도록 학습

### 4.2. 미세조정(Fine-tuning)

- 특정 작업(예: Q&A, 번역 등)에 적합하게 **파인튜닝**
- 최근에는 **지시어 학습(Instruct-tuning)** 또는 **RLHF**(Human Feedback 기반 보강) 방식 사용

## 5. **LLM의 활용 분야**

- **텍스트 생성**: 기사 작성, 소설, 보고서 등
- **요약 및 번역**: 전문 문서 요약, 다국어 번역
- **질문 응답**: 고객센터 챗봇, 정보 검색 시스템
- **코드 생성**: 프로그래밍 코드 자동 생성 및 오류 수정
- **지식 기반 구축**: 문서에서 지식 추출 및 자동 정리

## 6. **한계점과 과제**

| 항목 | 설명 |
| --- | --- |
| 비용 | 학습 및 추론에 막대한 연산 자원 소요 |
| 편향 | 훈련 데이터에 포함된 **사회적 편향** 반영 가능성 |
| 정확도 | 사실과 다른 정보를 **그럴듯하게 생성**하는 경향 |
| 제어 불가능성 | 예측 불가한 출력을 생성할 위험 존재 |

## 7. **요약**

- **LLM은 대규모 데이터를 기반으로 언어를 이해하고 생성하는 인공지능 모델**
- **Transformer 아키텍처** 기반으로 동작하며, **자연어 처리 전반을 자동화**하는 핵심 기술로 자리잡음
- 광범위한 응용 가능성과 동시에 기술적·윤리적 도전 과제도 존재함

# 

---

# LLM 기반 챗봇의 동작 과정

## 1. **입력 수집 (User Input Handling)**

- 사용자가 챗봇에 **자연어 문장**으로 질문을 입력함
- 예: "이번 주말 서울 날씨 알려줘"

 **관련 개념**:

LLM은 자연어로 입력된 문장을 **문맥(Context)**으로 처리함

## 2. **문맥 이해 및 전처리 (Preprocessing & Context Embedding)**

- 입력 문장을 **토큰(token)**으로 분해
    
    (예: “이번”, “주말”, “서울”, “날씨” → 각각의 단어/부분어)
    
- 토큰을 **벡터(embedding)**로 변환하여 모델 입력값으로 사용

**관련 개념**:

Transformer 기반 LLM은 **단어의 의미를 벡터 공간에서 표현**함

이 과정은 모델이 입력의 **의미와 문맥 관계**를 이해하는 핵심 절차임

## 3. **응답 생성 (Inference)**

- LLM은 입력된 문맥을 바탕으로, **다음에 올 단어 또는 문장**을 예측함
- 이는 **확률적으로 가장 적절한 출력 시퀀스를 생성**하는 과정임
- 생성된 응답은 하나의 문장 또는 복수의 문장일 수 있음

**관련 개념**:

LLM의 핵심은 **조건부 확률 기반 텍스트 생성**이며,

“이 문맥에서 가장 적절한 다음 단어는 무엇인가?”라는 질문에 수백억 개 파라미터로 답하는 구조임

## 4. **후처리 및 포맷팅 (Postprocessing)**

- 모델이 생성한 응답을 사람이 읽기 좋도록 **형식화**함
- 필요에 따라 특정 문장 정리, 이모티콘 제거, 줄 바꿈 등을 수행함
- 일부 챗봇은 응답에 **외부 정보(API 응답)**를 삽입하거나 조합함

**관련 개념**:

LLM은 생성형 모델이므로 **정형화된 응답 포맷이나 API 결과와의 결합**은 후처리 로직에서 주로 수행됨

## 5. **응답 출력 (Response Delivery)**

- 최종 응답이 사용자 인터페이스(UI)를 통해 출력됨
- 일반적으로 **채팅 창** 형태이며, **실시간 응답**이 이상적인 형태임

## 고급 챗봇에서는 다음과 같은 기능이 추가됨:

| 기능 | 설명 |
| --- | --- |
| 메모리 유지 | 이전 대화를 기억하고 연속적인 맥락 파악 |
| 도메인 지식 연동 | 법률, 의료 등 특정 도메인 지식을 포함 |
| 외부 도구 호출 | 날씨, 검색, 계산기, DB 조회 등 기능 수행 |
| 안전성 필터링 | 부적절하거나 유해한 콘텐츠를 제거 |

## 전체 흐름 요약

```markdown
[입력] → [문맥 임베딩] → [LLM 추론] → [후처리] → [출력]

```

## LLM과 챗봇의 관계

- **LLM은 챗봇의 두뇌**에 해당하며, 입력된 문장을 분석하고 그에 맞는 응답을 생성하는 **생성형 모델**임
- 챗봇은 LLM을 중심으로, 입력을 수집하고 출력하는 **인터페이스와 기능들을 조합한 시스템 구조**임

---

# RAG 아키텍처란?

## 1. RAG의 정의

**RAG**는 **Retrieval-Augmented Generation**의 약자로,

**검색 기반 생성형 아키텍처**를 의미함.

기존 LLM이 가지고 있는 고정된 지식만으로 답변을 생성하는 한계를 극복하기 위해,

**외부 지식베이스에서 관련 정보를 검색한 뒤 이를 바탕으로 응답을 생성하는 구조**를 가짐.

## 2. 등장 배경

- 일반적인 LLM은 훈련 시점 이후의 정보나 세부적 사실에 대해 **정확한 답변을 생성하지 못하는 한계**를 가짐
- 실시간 정보, 기업 내부 문서, 특정 도메인 데이터 등을 포함하기 위해 **검색(Search)** 모듈과 **생성(Generation)** 모듈의 결합이 필요해짐

## 3. RAG 아키텍처의 핵심 구성요소

| 구성 요소 | 설명 |
| --- | --- |
| 질의(Query) | 사용자가 입력한 자연어 질문 |
| 검색기(Retriever) | 사용자 질문과 관련된 외부 문서나 지식을 **벡터 기반 검색** 방식으로 찾아냄 |
| 문서 집합(External Knowledge Base) | 사전 구축된 문서 벡터 인덱스. PDF, 위키피디아, 보고서, DB 등 가능 |
| 생성기(Generator) | 검색된 문서를 기반으로 최종 답변을 **LLM이 생성**함 |

## 4. 동작 과정

### 1단계: 입력

- 사용자가 챗봇에 질문 입력
    
    예: “챗GPT는 어떻게 작동하나요?”
    

### 2단계: 벡터 변환 및 검색

- 입력 질문을 벡터로 임베딩하여, 벡터 인덱스 기반 검색기로부터 **관련 문서 Top-k**를 조회함

### 3단계: 문서 결합

- 검색된 문서들을 하나의 프롬프트로 통합하여 **LLM의 입력으로 사용**함

### 4단계: LLM 기반 응답 생성

- LLM은 검색된 문서 내용을 참조하여 최종 답변을 생성함
    
    (단순 예측이 아닌 **문서에 기반한 생성**)
    

## 5. 전통 LLM 방식과의 비교

| 항목 | 일반 LLM | RAG |
| --- | --- | --- |
| 지식 출처 | 사전 학습 데이터에 내재 | 외부 문서 기반 |
| 최신성 반영 | 불가능 | 가능 |
| 도메인 지식 반영 | 제한적 | 외부 문서로 쉽게 확장 가능 |
| 모델 크기 의존성 | 큼 | 상대적으로 줄일 수 있음 |

## 6. RAG의 장점

- **최신 정보 반영 가능**
    
    (예: 실시간 뉴스, 기업 내부 문서 등)
    
- **모델 파라미터를 재학습하지 않고도 지식 업데이트 가능**
- **사실성 향상**
    
    검색된 문서에 기반해 생성되므로 **헛소리(hallucination)** 현상 감소
    
- **도메인 특화 적용 용이**
    
    의료, 법률, 금융 등 특정 분야 문서를 인덱싱하여 적용 가능
    

## 7. 대표 응용 사례

- **기업용 AI 어시스턴트**: 내부 문서 검색 및 요약
- **리서치 보조 도구**: 논문 검색 후 핵심 내용 요약
- **법률·의료 챗봇**: 특정 판례, 논문, 매뉴얼에 근거한 정답 생성
- **고객센터**: FAQ 문서 기반 응답 자동화

## 8. 요약

- RAG는 LLM에 **검색(Search)** 기능을 결합하여 **보다 정확하고 시의성 있는 응답을 생성**하는 구조임
- 단순 생성형 모델이 아닌, **외부 정보에 기반한 생성 모델**이라는 점에서 큰 강점을 가짐
- 실제 현업에서 **대규모 LLM을 효율적으로 활용하기 위한 핵심 아키텍처로 각광받고 있음**

---

# Prompt Engineering이란?

## 1. Prompt의 의미

**Prompt**는 **LLM에게 주어지는 입력 문장 또는 지시문**을 의미함.

사용자가 “무엇을, 어떤 방식으로” 묻는지를 결정하는 핵심 요소임.

예시:

- “AI가 무엇인지 설명해줘”
- “다음 문장을 세 문장으로 요약해줘: …”
- “아래 코드를 설명해줘: …”

## 2. Prompt Engineering의 정의

**Prompt Engineering**은

**LLM의 출력을 원하는 방향으로 유도하기 위해 입력(Prompt)을 정교하게 설계하는 기술적 기법**을 의미함.

단순히 문장을 입력하는 것이 아니라,

**문장의 구조, 어휘 선택, 조건 제시 방식** 등을 세심하게 설계하여 **정확하고 일관된 출력**을 유도하는 작업임.

## 3. 등장 배경

- LLM은 기본적으로 **입력된 문장에 대한 확률적 텍스트 생성**을 수행함
- 동일한 질문도 **어떻게 묻느냐에 따라 답변이 달라지는 현상**이 빈번하게 발생함
- 따라서 **정확하고 일관된 결과를 얻기 위해서는 Prompt 자체를 엔지니어링할 필요성**이 대두됨

## 4. Prompt Engineering의 핵심 목표

| 목표 | 설명 |
| --- | --- |
| **정확성 향상** | 애매하거나 잘못된 답변을 줄이기 위함 |
| **일관성 확보** | 유사 질문에 대해 동일한 형식과 품질의 응답을 얻기 위함 |
| **작업 최적화** | 요약, 번역, 코드 생성 등 특정 작업을 자동화하기 위함 |
| **모델 제한 보완** | 모델이 학습하지 않은 내용도 간접적으로 유도 가능 |

## 5. 주요 기법

### 5.1. 명령형 프롬프트 (Instruction Prompting)

- “~을 수행해라”와 같이 **명확한 지시문**을 사용
    
    예: “이 문장을 한 줄 요약해줘”
    

### 5.2. Few-shot Prompting

- 예시를 몇 개 포함하여 **출력 형식을 유도**
    
    예:
    
    ```makefile
    예시1: 입력 → 출력
    예시2: 입력 → 출력
    새로운 입력: … → ?
    
    ```
    

### 5.3. Zero-shot Prompting

- 예시 없이 **직접 명령만으로 수행**
    
    예: “이 문장을 영어로 번역해줘”
    

### 5.4. Chain-of-Thought Prompting

- **논리적인 사고 과정을 유도**
    
    예: “단계별로 생각하고 답을 도출해줘”
    

### 5.5. System Prompting (Role Prompting)

- LLM에게 특정 역할을 부여하여 출력을 제어
    
    예: “당신은 법률 전문가입니다. 아래 계약서를 검토해 주세요.”
    

## 6. Prompt Engineering의 중요성

- **LLM의 동작은 고정된 파라미터에 기반**하므로,
    
    모델 자체를 수정하지 않고 출력을 조절하는 가장 효과적인 방법이 바로 Prompt 설계임
    
- 특히 **RAG 아키텍처**에서는 검색된 문서를 함께 넣는 Prompt를 어떻게 구성하느냐에 따라 응답 품질이 크게 달라짐
- LLM을 API로 사용하는 기업, 연구기관, 개발자는 Prompt Engineering을 통해
    
    **비용 대비 정확도를 최적화**할 수 있음
    

## 7. 실무 활용 예시

| 분야 | Prompt Engineering 예 |
| --- | --- |
| 고객 응대 | “다음 고객 문의에 친절하게 3문장으로 답변해줘…” |
| 코드 생성 | “파이썬으로 이진 탐색 알고리즘을 구현해줘” |
| 법률 | “다음 조항에 대해 위반 요소가 있는지 판례를 기반으로 분석해줘” |
| 요약 | “다음 기사 내용을 두 문단 요약으로 정리해줘” |

## 8. 한계 및 도전 과제

- **출력의 불확실성**: 동일 프롬프트에서도 결과가 달라질 수 있음
- **길이 제한**: 프롬프트와 응답이 모델의 입력 토큰 한계를 초과할 수 있음
- **프롬프트 실험의 반복**: 원하는 출력을 얻기까지 반복적 튜닝이 필요함

## 9. 요약

- Prompt Engineering은 LLM 기반 시스템에서 **출력을 효과적으로 제어하고 최적화하는 핵심 기술**임
- 명령형, 예시 기반, 역할 부여 등 다양한 기법을 통해 정확성과 일관성을 높일 수 있음
- RAG와 결합 시, 검색된 문서를 어떻게 Prompt에 삽입하느냐가 최종 응답 품질에 결정적 영향을 미침

---

# 우리 프로젝트를 클라우드(AWS)에 배포하기 위해서 어떤 과정을 거쳐야 하는가?

# 목표

**Gradio + RAG 아키텍처 기반 챗봇**을

**Docker로 패키징하고, AWS EC2에 배포**하여

누구나 웹에서 컴퓨터공학과 정보를 질문하고 답변받을 수 있도록 구현하는 것

## 1. 사전 준비물 요약

| 항목 | 설명 |
| --- | --- |
| AWS 계정 | EC2 인스턴스를 만들기 위해 필요 |
| EC2 인스턴스 | 챗봇 서버를 구동할 리눅스 가상머신 (Ubuntu 22.04 권장) |
| PEM 키 | EC2에 SSH 접속하기 위한 인증 키 |
| Dockerfile | 챗봇을 도커로 감싸기 위한 설정 파일 |
| requirements.txt | 챗봇 실행에 필요한 라이브러리 목록 |
| main.py | Gradio UI와 RAG 챗봇 동작을 포함한 메인 코드 |

## 2. 프로젝트 구조 예시

```
plaintext
복사편집
rag-chatbot/
├── main.py              # Gradio 기반 챗봇 인터페이스 + RAG 로직
├── retriever.py         # 문서 검색기 로직
├── generator.py         # LLM 응답 생성 로직
├── data/                # 학과 정보 문서 모음 (텍스트, PDF 등)
├── vector_index/        # 벡터화된 문서 인덱스 (FAISS 등)
├── Dockerfile           # 컨테이너 구성 파일
├── requirements.txt     # 필요한 파이썬 패키지 목록

```

## 3. Dockerfile 예시 (기본형)

```dockerfile

FROM python:3.10

# 작업 디렉토리 설정
WORKDIR /app

# 로컬 파일 복사
COPY . .

# 라이브러리 설치
RUN pip install --no-cache-dir -r requirements.txt

# Gradio 실행 (포트 지정)
CMD ["python", "main.py"]

```

## 4. EC2에 배포하는 전체 절차

### 1단계: EC2 인스턴스 생성

- AWS 콘솔 → EC2 → 인스턴스 시작
- Ubuntu 22.04 선택
- t2.micro 또는 t3.small 선택 (테스트용이면 무료 범위)
- 포트 규칙 설정:
    - 22번 (SSH)
    - 7860번 (Gradio 웹)
    - 80번 (원하는 경우 HTTP 웹 접근)

### 2단계: EC2 서버 접속

```bash
chmod 400 your-key.pem
ssh -i your-key.pem ubuntu@<EC2-퍼블릭-IP>

```

### 3단계: 서버에 Docker 설치

```bash
# 시스템 업데이트
sudo apt update && sudo apt upgrade -y

# Docker 설치
sudo apt install docker.io -y

# Docker 권한 설정 (옵션)
sudo usermod -aG docker ubuntu
newgrp docker

```

### 4단계: 챗봇 코드 서버에 업로드

방법 1: GitHub에서 clone

```bash
git clone https://github.com/your-username/rag-chatbot.git
cd rag-chatbot

```

방법 2: 로컬에서 EC2로 파일 전송

```bash
scp -i your-key.pem -r ./rag-chatbot ubuntu@<EC2-IP>:~/rag-chatbot

```

### 5단계: Docker 이미지 빌드 및 실행

```bash
# 빌드
docker build -t rag-chatbot .

# 실행
docker run -d -p 7860:7860 rag-chatbot

```

※ `main.py`에 다음과 같이 작성되어 있어야 외부 접속 가능:

```python
gr.Interface(...).launch(server_name="0.0.0.0", server_port=7860)

```

### 6단계: 접속 테스트

- 브라우저에서 접속
    
    ```cpp
    http://<EC2 퍼블릭 IP>:7860
    
    ```
    
- 챗봇 UI가 나타나고, 질문 입력 시 실제 문서를 기반으로 LLM이 응답

## 5. 이후 고려할 수 있는 작업

| 작업 항목 | 설명 |
| --- | --- |
| `.env` 파일 관리 | OpenAI API 키, 프롬프트 설정 등을 환경변수로 분리 |
| 도커 이미지 백업 | `docker save`를 사용하여 이미지 저장 가능 |
| HTTPS 적용 | Nginx + Let's Encrypt를 사용하여 SSL 인증 적용 가능 |
| 자동 실행 설정 | EC2 부팅 시 자동 실행되도록 `systemd` 또는 `docker-compose` 설정 가능 |

## 6. 전체 요약

| 단계 | 설명 |
| --- | --- |
| 개발 | Gradio + RAG 챗봇을 하나의 Python 코드로 구현 |
| 컨테이너화 | Dockerfile로 애플리케이션 환경 패키징 |
| 배포 | EC2 인스턴스에 도커 설치 후 이미지 빌드 및 실행 |
| 운영 | 퍼블릭 IP로 사용자 접속 가능, 챗봇 실시간 응답 제공 |